---
title: Fasion MNIST Convolutional Neural Network
description: Define the CNN for prection of Fasion MNIST datasets
date: 2025-02-08 23:11
author: Holmes Amzish
tags: ['machine-learning', 'cnn', 'pytorch']
---

In [28]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import matplotlib
import matplotlib.pyplot as plt
import os
import random

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
matplotlib.use('TkAgg')

In [29]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_data = datasets.FashionMNIST(root='data', train=True, download=True, transform=transform)
test_data = datasets.FashionMNIST(root='data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

In [30]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)

        # Pooling layers
        self.pool = nn.MaxPool2d(2, 2)

        # Full connected layers
        self.fc1 = nn.Linear(128 * 3 * 3, 512)
        self.fc2 = nn.Linear(512, 10)

        # Dropout layer
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))

        x = x.view(-1, 128 * 3 * 3)

        x = torch.relu(self.fc1(x))
        x = self.dropout(x)

        x = self.fc2(x)
        return x

In [31]:
model = NeuralNetwork()
print(model)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")
model.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


NeuralNetwork(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1152, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)
Using cuda device


In [32]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Computer prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {100*(correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")

In [33]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, loss_fn, optimizer)
    test(test_loader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302333  [   64/60000]
loss: 0.647033  [ 6464/60000]
loss: 0.580490  [12864/60000]
loss: 0.485724  [19264/60000]
loss: 0.545703  [25664/60000]
loss: 0.477990  [32064/60000]
loss: 0.333280  [38464/60000]
loss: 0.251924  [44864/60000]
loss: 0.278290  [51264/60000]
loss: 0.322126  [57664/60000]
Test Error: 
 Accuracy: 88.1%, Avg loss: 0.323977

Epoch 2
-------------------------------
loss: 0.460136  [   64/60000]
loss: 0.192585  [ 6464/60000]
loss: 0.198050  [12864/60000]
loss: 0.388034  [19264/60000]
loss: 0.303517  [25664/60000]
loss: 0.274965  [32064/60000]
loss: 0.389794  [38464/60000]
loss: 0.321096  [44864/60000]
loss: 0.327351  [51264/60000]
loss: 0.432855  [57664/60000]
Test Error: 
 Accuracy: 89.7%, Avg loss: 0.277630

Epoch 3
-------------------------------
loss: 0.248235  [   64/60000]
loss: 0.171419  [ 6464/60000]
loss: 0.281354  [12864/60000]
loss: 0.211329  [19264/60000]
loss: 0.168916  [25664/60000]
loss: 0.373718  [32064/60000

In [ ]:
# Save current model
torch.save(model.state_dict(), "models/fashion_mnist_model.pth")
print("Saved PyTorch Model State to fashion_mnist_model.pth")

In [ ]:
# Load model from pth file
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("models/fashion_mnist_model.pth"))

In [53]:
model.eval()

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

random_index = random.randint(0, len(test_data))
x, y = test_data[random_index][0], test_data[random_index][1]
with torch.no_grad():
    x = x.unsqueeze(0).to(device)
    pred = model(x)
    predicted_class = pred.argmax(1).item()

plt.imshow(x.squeeze().cpu(), cmap="gray")
plt.title(f"Predicted: {classes[predicted_class]}, Actual: {classes[y]}")
plt.show()